<a href="https://colab.research.google.com/github/moonryul/course-v3/blob/master/QuadCurveGanFinalExam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install keras

In [ ]:
# train a generative adversarial network on a one-dimensional function
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot


# define the standalone discriminator model
def define_discriminator(n_inputs=2):
    # class Sequential(Model) = Linear stack of layers
    # The `Model` class adds training & evaluation routines to a `Network`.
    # class Model(Network): add(self, layer): 	Adds a layer instance on top of the layer stack.

    model = Sequential()  # model is an object of class Sequential
    model.add(Dense(25, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
    # n_inputs = 2; n_output=25:  The Input vector is a 2D point (x,y)
    model.add(Dense(1, activation='sigmoid'))
    # n_input = 25 = n_output of the previous layer; n_output =1 ( The value of the discriminator output is probability between  o and 1
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # model.loss = loss ="binary_crossentropy": Using the binary cross entropy function as the loss
    # function means that it tries to minimize the difference (loss) between the predicted probability of the net
    # for the input to belong to one of the two categories and the labeled probability which is either 1 (real)or 0 (fake).
    # It tries to minimize this difference (loss) for all inputs (whose number is 128 in our example); it means that
    # it tries ot minimize the average difference (loss) of the all inputs. The basic idea is the same with the mean square error
    # loss for regression problem. The difference is the "value" used to compute the loss probability rather than ordinary values.

    return model  # model is a reference  to the current instance of the class


# define the standalone generator model
def define_generator(latent_dim, n_outputs=2):  # latent_dim =5
    model = Sequential()
    model.add(Dense(15, activation='relu', kernel_initializer='he_uniform',
                    input_dim=latent_dim))  # n_input=5 = latent_dim; n_output=15
    model.add(Dense(n_outputs, activation='linear'))  # n_input = 15 = n_output of the previous layer; n_output = 2
    # The dimension of the output of the geneator is 2, because it generates a 2D point (x,y) which is supposed to lie on
    # the quadratic curve y = x^2;
    return model


# define the combined generator and discriminator model for updating the generator
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False  # discriminator is set as not trainable when it is part of the composite model
    # But it is trainable when it is used alone
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam')
    # model.loss = loss ="binary_crossentropy"
    return model

#  Making the discriminator not trainable is a clever trick in the Keras API. The trainable
#  property impacts the model when it is compiled. The discriminator model was compiled with
#  trainable layers, therefore the model weights in those layers will be updated when the standalone
#  model is updated via calls to train on batch().

# generate n real samples with class labels "ones" for training the discriminator
def generate_real_samples(n): # n = 128/2
    # generate inputs in [-0.5, 0.5]
    X1 = rand(n) - 0.5
    # generate outputs X^2
    X2 = X1 * X1
    # stack arrays
    X1 = X1.reshape(n, 1)
    X2 = X2.reshape(n, 1)
    X = hstack((X1, X2))  # X =  hstack( [1,2], [3,4] ) ==>[ [1,3],[2,4] ] : 128 points
    # generate class labels
    y = ones((n, 1))  # y = 128/2 labels
    return X, y  # # A pair of 128/2 real samples and their 128 labels


# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n):
    # generate points in the latent space
    x_input = randn(latent_dim * n)  # [01, 02, 0.9,...., 0,1]
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n, latent_dim)  # 128 * 5 matrix
    return x_input


# use the generator to generate n fake examples, with class labels "zero", for training the discriminator
def generate_fake_samples(generator, latent_dim, n): # n = 128/2
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n)  # 128/2  x 5: 128/2 samples of 5 random numbers
    # predict outputs
    X = generator.predict(x_input)  # X = 128/2 generator outputs for 128/2 samples of 5 numbers
    # create class labels
    y = zeros((n, 1))  # y = 128/2  labels
    return X, y  # A pair of 128/2 fake samples and their 128/2 labels


# evaluate the discriminator and plot real and fake points
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):
    # prepare real samples
    x_real, y_real = generate_real_samples(n)  # (x_real, y_real):  A pair of 128 real samples and their 128 labels
    # evaluate discriminator on real examples
    _, acc_real = discriminator.evaluate(x_real, y_real,
                                         verbose=0)  # acc_real = THe accuray of the discriminator net that tells "real" for real samples (inputs)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(generator, latent_dim,
                                           n)  # (x_fake, y_fake):  # A pair of 128 fake samples and their 128 labels
    # evaluate discriminator on fake examples
    _, acc_fake = discriminator.evaluate(x_fake, y_fake,
                                         verbose=0)  # acc_fake = The accuray of the discriminator net that tells "fake" for fake samples (inputs)
    # summarize discriminator performance
    print(epoch, acc_real, acc_fake)  # print both acc_real and acc_fake for the current epoch.
    # scatter plot real and fake data points: 
    # x_real[i,0] is the x coord of ith real point and x_real[i,1] is the y coord of ith real point

    pyplot.scatter(x_real[:, 0], x_real[:, 1], color='red')
    pyplot.scatter(x_fake[:, 0], x_fake[:, 1], color='blue')
    # save plot to file
    filename = 'generated_plot_e%03d.png' % (epoch + 1)
    pyplot.savefig(filename)
    pyplot.close()


# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=10000, n_batch=128, n_eval=2000):
    # determine half the size of one batch, for updating the discriminator
    half_batch = int(n_batch / 2)

    # loop over epochs: In the case of typical supervised learning, one epoch refers to one scan over the entire dataset
    # in the process of training the network.
    # In this gan example, one epoch refers to one scan over a single mini-batch.
    # This is because the real data is prepared not in the form of entire dataset in the beginning,
    # but is computed in the form of mini-batch by "generate_real_samples" function at each update of the discriminator.
    #
    for i in range(n_epochs):
        # 1) Train the discriminator to discriminate between real-data and fake-data

        # 1.1)  prepare real samples (real mini-batch): note that in this example, the discriminator will see
        #  10000 * 128/2 real data ( 2D points from the quadratic curve y = x^2) in total throughout all the epochs.
        # It means also that the discriminator encounters the same number of fake 2D points throughout all the epochs.
        # and is trained to tell "fake" to them. The fake data used by the discriminator become smarter as the epoch
        # progresses.

        x_real, y_real = generate_real_samples(half_batch)
        # prepare fake examples (fake mini_batch)
        x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        # 1.2)  update discriminator
        # "Runs a single gradient update on a single mini-batch of data":
        d_model.train_on_batch(x_real, y_real)  # train the discriminator using the real mini-batch
        d_model.train_on_batch(x_fake,y_fake)  # train the discriminator using the fake mini-batch.

        # 2) Train the generator net
        # 2.1) prepare points in latent space as input for the generator
        x_gan = generate_latent_points(latent_dim,
                                       n_batch)  # x_gan = 128 x 5 matrix; This means 128 samples of 5 random numbers
        # 2.2) create labels that the discriminator should produce for the fake sample output of the generator
        # Note that while the generator is updated in order to "fool" the discriminator, the discriminator is fixed.
        y_gan = ones((n_batch, 1))  # 128 1's

        # 2.3) update the generator  so that the loss of the fixed discriminator is minimized.
        gan_model.train_on_batch(x_gan, y_gan)   # x_gan= input to the generator = 128 samples of 5 random vectors,
                                                 # y_gan = target label = 128 labels (all 1's) for the discriminator

        # NOTE: gan_model.train_on_batch(x_gan, y_gan) tries to train  the generator so that the discriminator tells "real"
        # for EVERY (fake) output generated by the generator. Note that this goal is a bit indirect in the sense that
        # the generator does not attempt to optimize its output directly but it tries to optimize the
        # the output of the discriminator. But it is not uncommon. For example, consider the effort of parents
        # with respect to their children.

        # evaluate the model every n_eval epochs
        if (i + 1) % n_eval == 0:
            summarize_performance(i, g_model, d_model, latent_dim)




In [ ]:
# size of the latent space (the size of a random input vector to the generator)
latent_dim = 5
# create the discriminator
discriminator = define_discriminator()
#  discriminator is a reference  to the instance of the Sequential class
#  discriminator defines the loss function and the optimization method

# create the generator
generator = define_generator(latent_dim)  # generator does not define  the loss function and the optimization method
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim, n_epochs=10000, n_batch=128, n_eval=1000)
# 1) train the discriminator on real samples and the fake samples generated by the current generator net
# 2) Then, train the generator with the discriminator set frozen (not trainable)


999 0.49000000953674316 0.44999998807907104
1999 0.47999998927116394 0.5600000023841858
2999 0.6399999856948853 0.4300000071525574
3999 0.4300000071525574 0.6499999761581421
4999 0.5400000214576721 0.5699999928474426
5999 0.8199999928474426 0.3700000047683716
6999 0.7699999809265137 0.5199999809265137
7999 0.7300000190734863 0.3499999940395355
8999 0.6100000143051147 0.47999998927116394
9999 0.7099999785423279 0.4399999976158142
